<a href="https://colab.research.google.com/github/rudranshagarwal/RadChest-CT/blob/main/slic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SimpleITK

In [ ]:
!pip install pynrrd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk

from skimage.segmentation import slic, quickshift, felzenszwalb
from skimage.segmentation import mark_boundaries
from skimage.measure import regionprops
from skimage.util import img_as_float
from skimage import io, exposure, util

# from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import pickle
import glob
import sys
import nrrd

In [ ]:
def read_mha_file(filepath):
    """Read and load volume"""
    # Read file
    scan_sitk = sitk.ReadImage(filepath, sitk.sitkFloat32)
    # Get raw data
    scan = sitk.GetArrayFromImage(scan_sitk)
    return scan

def read_mask_file(filepath):
    """Read and load volume"""
    # Read file
    mask_sitk = sitk.ReadImage(filepath, sitk.sitkInt16)
    # Get raw data
    mask = sitk.GetArrayFromImage(mask_sitk)
    return mask

def normalize(volume):
    """Normalize the volume"""
    min = -1350
    max = 150
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_mha_file(path)
    # Normalize
    volume = normalize(volume)
    return volume

# print("\nSupervoxelation for files from: {} to: {}\n".format(file_start, file_end))

# for k in range(file_start, file_end):


scanpath = "/content/drive/MyDrive/output_image.nii"
maskpath = "/content/drive/MyDrive/test.nii"

print("\nProcessing file: {} and mask: {}\n".format(scanpath, maskpath))

scan = process_scan(scanpath)
mask = read_mask_file(maskpath)


def super_voxelation(scan, mask):

    num_supervoxels = 10

    num_query = 12
    num_extracted = 11

    compactness = 0.9

    count = 0
    print("here")
    while num_extracted != num_supervoxels:

        count += 1

        if count >= 6:
            print("No solution in 6 iterations for {}.".format("test"))
            break

        segments_slic = slic(scan, compactness=compactness, sigma=0, enforce_connectivity=True,
                    n_segments=num_query, start_label=1, mask=mask, channel_axis=None)
        print(segments_slic)


        indices = np.unique(segments_slic)
        num_extracted = len(indices)

        print("Number of query supervoxels: {} Number of extracted superpixels: {}".format(num_query, num_extracted))

        if num_extracted > num_supervoxels:
            num_query -=1
        elif num_extracted < num_supervoxels:
            num_query +=1
        else:
            if num_query == num_extracted:
                compactness -=0.1
            else:
                num_query +=1

    slic_segmentation = np.swapaxes(segments_slic, 0, 2)
    return slic_segmentation



Processing file: /content/drive/MyDrive/output_image.nii and mask: /content/drive/MyDrive/test.nii



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mask = read_mask_file(maskpath)

slic_segmentation = super_voxelation(mask,scan)
nrrd.write('/content/drive/MyDrive/' + str(f'test_slic') + '.nrrd', slic_segmentation)

here
[[[ 1  1  1 ...  2  2  2]
  [ 1  1  1 ...  2  2  2]
  [ 1  1  1 ...  2  2  2]
  ...
  [ 4  4  4 ...  5  5  5]
  [ 4  4  4 ...  5  5  5]
  [ 4  4  4 ...  5  5  5]]

 [[ 1  1  1 ...  2  2  2]
  [ 1  1  1 ...  2  2  2]
  [ 1  1  1 ...  2  2  2]
  ...
  [ 4  4  4 ...  5  5  5]
  [ 4  4  4 ...  5  5  5]
  [ 4  4  4 ...  5  5  5]]

 [[ 1  1  1 ...  2  2  2]
  [ 1  1  1 ...  2  2  2]
  [ 1  1  1 ...  2  2  2]
  ...
  [ 4  4  4 ...  5  5  5]
  [ 4  4  4 ...  5  5  5]
  [ 4  4  4 ...  5  5  5]]

 ...

 [[11 11 11 ...  9  9  9]
  [11 11 11 ...  9  9  9]
  [11 11 11 ...  9  9  9]
  ...
  [12 12 12 ... 10 10 10]
  [12 12 12 ... 10 10 10]
  [12 12 12 ... 10 10 10]]

 [[11 11 11 ...  9  9  9]
  [11 11 11 ...  9  9  9]
  [11 11 11 ...  9  9  9]
  ...
  [12 12 12 ... 10 10 10]
  [12 12 12 ... 10 10 10]
  [12 12 12 ... 10 10 10]]

 [[11 11 11 ...  9  9  9]
  [11 11 11 ...  9  9  9]
  [11 11 11 ...  9  9  9]
  ...
  [12 12 12 ... 10 10 10]
  [12 12 12 ... 10 10 10]
  [12 12 12 ... 10 10 10]]]
Numbe

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import nibabel as nib

from skimage.segmentation import slic, quickshift, felzenszwalb
from skimage.segmentation import mark_boundaries
from skimage.measure import regionprops
from skimage.util import img_as_float
from skimage import io, exposure, util

# from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import pickle
import glob
import sys
import nrrd

# file_start = int(sys.argv[1])
# file_end = int(sys.argv[2])

# files = glob.glob("ctrs/*.mha")

def read_mha_file(filepath):
    """Read and load volume"""
    # Read file
    scan_sitk = sitk.ReadImage(filepath, sitk.sitkFloat32)
    # Get raw data
    scan = sitk.GetArrayFromImage(scan_sitk)
    # scan = nib.load(filepath).get_fdata()
    return scan

def read_mask_file(filepath, label):
    """Read and load volume"""
    # Read file
    mask_sitk = sitk.ReadImage(filepath, sitk.sitkInt16)
    # Get raw data
    mask = sitk.GetArrayFromImage(mask_sitk)
    # mask = nib.load(filepath).get_fdata()
    new_mask = (mask == label)
    return  (mask == label)

def normalize(volume):
    """Normalize the volume"""
    min = -1350
    max = 150
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_mha_file(path)
    # Normalize
    # volume = normalize(volume)
    return volume

# print("\nSupervoxelation for files from: {} to: {}\n".format(file_start, file_end))

# for k in range(file_start, file_end):

#     file = files[k][5:]

scanpath = "/content/drive/MyDrive/output_image.nii"
maskpath = "/content/drive/MyDrive/test.nii"

print("\nProcessing file: {} and mask: {}\n".format(scanpath, maskpath))

scan = process_scan(scanpath)
final_scan = np.zeros(scan.shape)
subsegments = {
    1: 10,
    2: 10,
    3: 6,
    4: 4,
    5: 12
}
n_segments_done = 0
segments = []
for label in range(1,6):
    mask = read_mask_file(maskpath,label)
    segments_slic = []
    num_supervoxels = subsegments[label] + 1
    flag = 1
    num_query = subsegments[label] + 2
    num_extracted = subsegments[label] + 3

    compactness = 0.9

    count = 0

    while num_extracted != num_supervoxels:

        count += 1
        print(count, label)
        if count > 6:
            print("No solution in 6 iterations for {}.")
            flag = 0
            break

        segments_slic = slic(scan, compactness=compactness, sigma=0, enforce_connectivity=True,
                    n_segments=num_query, start_label=1, mask=mask, channel_axis=None)

        indices = np.unique(segments_slic)
        num_extracted = len(indices) - 1

        print("Number of query supervoxels: {} Number of extracted superpixels: {}".format(num_query, num_extracted))

        if num_extracted > num_supervoxels:
            num_query -=1
        elif num_extracted < num_supervoxels:
            num_query +=1
        else:
            if num_query == num_extracted:
                compactness -=0.1
            else:
                num_query +=1
        print(np.unique(segments_slic, return_counts=True))
    slic_segmentation = np.swapaxes(segments_slic, 0, 2)

    slic_segmentation[np.where(slic_segmentation>0)] += n_segments_done
    slic_segmentation[np.where(slic_segmentation == n_segments_done + num_extracted)] = 0
    print(n_segments_done)
    n_segments_done +=num_extracted - 1
    print(n_segments_done)
    final_scan += slic_segmentation
    print(np.unique(final_scan))
print(segments)
nrrd.write('/content/drive/MyDrive/' + str(f'test_slic') + '.nrrd', final_scan)
    # np.save('/ssd_scratch/cvit/chocolite/data/slic_small/' + str(file), segments_slic)

    # centroids = []

    # centroids_1 = []
    # centroids_2 = []

    # regions = regionprops(segments_slic)

    # for props in regions:
    #     cx, cy, cz = props.centroid
    #     cx = int(cx)
    #     cy = int(cy)
    #     cz = int(cz)
    #     # print(cx, cy, cz)

    #     centroids.append([cx, cy, cz])

    #     # print(mask[cx, cy, cz] == 1)

    #     if mask[cx, cy, cz] == 1:
    #         centroids_1.append([cx, cy, cz])
    #     elif mask[cx, cy, cz] == 2:
    #         centroids_2.append([cx, cy, cz])

    # centroids = np.asarray(centroids)

    # from sklearn.neighbors import kneighbors_graph

    # A = kneighbors_graph(centroids, 3, mode='distance', include_self=False)
    # A.toarray()

    # import networkx as nx

    # G = nx.from_numpy_matrix(np.matrix(A.toarray()), create_using=nx.DiGraph)

    # layout = nx.spring_layout(G)




Processing file: /content/drive/MyDrive/output_image.nii and mask: /content/drive/MyDrive/test.nii

1 1
Number of query supervoxels: 12 Number of extracted superpixels: 13
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([75574416,   137310,   112471,   149566,   152221,   141568,
         155395,   100077,   138144,   128399,   136592,   119532,
         106719,       90]))
2 1
Number of query supervoxels: 11 Number of extracted superpixels: 12
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]), array([75574416,   137767,   141017,   159127,   150905,   150100,
         146106,   111002,   132143,   182224,   176848,    90737,
            108]))
3 1
Number of query supervoxels: 10 Number of extracted superpixels: 11
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([75574416,   143791,   140618,   182848,   155849,   191610,
         149382,   122575,   177157,   120622,   193524,      108]))
0
10
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.

In [ ]:
unique, counts = np.unique(final_scan,return_counts=True)

In [ ]:
print(counts)

[69975204   143791   140618   182848   155849   191610   149382   122575
   177157   120622   193524   149270   191399   185456   194438   162409
   172004   239258   163553   126221   145502   298448   336916   279367
   318643   292073   266665   178371   135379   166598   207193   113734
   119676   122377    98176   132414   118365   120138    98590   115898
   119852   113405   117532]


In [ ]:
print(final_scan.shape[0] *final_scan.shape[1] * final_scan.shape[2])

77152500


In [ ]:
mask = (final_scan == 11).astype(final_scan.dtype)
nrrd.write('/content/drive/MyDrive/' + str(f'test_slic') + '.nrrd', mask)


In [ ]:
print(np.where(final_scan == 11))

(array([262, 262, 335, 335, 336, 336, 366, 367, 372, 372]), array([124, 124, 122, 122, 122, 122, 169, 169, 145, 146]), array([301, 302, 322, 323, 322, 323, 287, 287, 274, 274]))


In [ ]:
import nibabel as nib

In [ ]:
# mask_sitk = sitk.ReadImage('/content/drive/MyDrive/test.nii', sitk.sitkInt16)
testmask = nib.load('/content/drive/MyDrive/test.nii').get_fdata()
print(np.unique(testmask))
print(np.unique(final_scan[np.where(testmask == 2)]))

[0. 1. 2. 3. 4. 5.]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


In [ ]:
print(np.where(mask >0))

(array([175, 176, 176, ..., 250, 250, 250]), array([289, 284, 285, ..., 222, 223, 271]), array([372, 373, 372, ..., 381, 381, 385]))


In [ ]:
print(np.where(testmask ==2  ))

(array([136, 136, 136, ..., 338, 338, 338]), array([253, 253, 267, ..., 146, 146, 146]), array([328, 329, 331, ..., 302, 303, 304]))


In [ ]:
print(final_scan[136][253][328])

11.0


In [ ]:
print(np.where(mask == 1))

(array([136, 136, 137, 137, 138, 138, 139, 139, 139, 139, 140, 140, 140,
       140, 141, 141, 141, 141, 173, 173, 208, 208, 209, 209, 213, 213,
       214, 214, 215, 215, 219, 220, 238, 238, 239, 239, 239, 240, 240,
       240, 246, 246, 247, 247, 248, 248, 251, 251, 252, 252, 253, 253,
       274, 274, 287, 287, 322, 322, 323, 323]), array([253, 253, 253, 253, 251, 252, 249, 249, 250, 250, 247, 247, 248,
       248, 247, 247, 248, 248, 223, 223, 163, 163, 162, 162, 162, 162,
       161, 161, 160, 160, 161, 160, 145, 145, 145, 145, 162, 145, 145,
       161, 166, 166, 166, 166, 165, 165, 162, 162, 162, 162, 161, 161,
       145, 146, 169, 169, 122, 122, 122, 122]), array([328, 329, 328, 329, 330, 330, 331, 332, 331, 332, 333, 334, 333,
       334, 333, 334, 333, 334, 363, 364, 375, 376, 375, 376, 377, 378,
       377, 378, 377, 378, 379, 379, 377, 378, 377, 378, 379, 377, 378,
       379, 375, 376, 375, 376, 375, 376, 375, 376, 375, 376, 375, 376,
       372, 372, 366, 367, 335, 336, 

In [ ]:
unique,count = np.unique(slic_segmentation, return_counts=True)

In [ ]:
print(np.where(final_scan == 10))

(array([175, 176, 176, ..., 250, 250, 250]), array([289, 284, 285, ..., 222, 223, 271]), array([372, 373, 372, ..., 381, 381, 385]))


In [ ]:
print(count, unique)

[75422930   149270   191399   185456   194438   162409   172004   239258
   163553   126221   145502       60] [ 0  1  2  3  4  5  6  7  8  9 10 11]


In [ ]:
print(np.where(testmask == 2))

(array([237, 237, 237, ..., 387, 387, 387]), array([278, 278, 278, ..., 285, 285, 285]), array([257, 258, 259, ..., 232, 233, 234]))


In [ ]:
print(final_scan[237][278][257])

0.0


In [ ]:
print(np.unique(testmask[mask == False]))

[0 1 3 4 5]


In [ ]:
print(np.unique(final_scan).shape)

(43,)


In [ ]:
array = np.array([[0,1],[1,2],[2,3],[0,1]])

array[np.where(array >0 )] += 2
print(array)

[[0 3]
 [3 4]
 [4 5]
 [0 3]]
